In [30]:
import torch
from torch import tensor, nn
from loading_datas import  generate_pair_sets
import torch.nn.functional as F



In [31]:

train_pairs, train_target, train_classes, test_pairs, test_target, test_classes = generate_pair_sets(1000)
print(train_target)


tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [1., 0.]])


In [32]:
# LeNET
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor1 = nn.Sequential(
            nn.Conv2d(in_channels=2, out_channels=6, kernel_size=3, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )

        self.feature_extractor2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )

        self.feature_extractor3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=2, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x1 = self.feature_extractor1(x)
        # print(x1.shape)
        x2 = self.feature_extractor2(x1)
        # print(x2.shape)
        x3 = self.feature_extractor3(x2)
        # print(x3.shape)
        x = torch.flatten(x3, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs
 

In [4]:
model = LeNet5(2)
output = model(train_pairs)
print(output[1])

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x216 and 6x2)

In [33]:
lr, nb_epochs, batch_size = 1e-1, 1000, 1000
model = LeNet5(2)
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()
for e in range(nb_epochs):
    for input, targets in zip(train_pairs.split(batch_size), train_target.split(batch_size)):
        output = model(input)[1]       
        # print(loss)
        loss= criterion(output, targets)
        optimizer.zero_grad()
        loss.backward()
        
    optimizer.step()

(tensor([[0.1106, 0.0842],
        [0.1141, 0.1162],
        [0.0766, 0.1047],
        ...,
        [0.0951, 0.0600],
        [0.1068, 0.0898],
        [0.1457, 0.0505]], grad_fn=<AddmmBackward0>), tensor([[0.5066, 0.4934],
        [0.4995, 0.5005],
        [0.4930, 0.5070],
        ...,
        [0.5088, 0.4912],
        [0.5043, 0.4957],
        [0.5238, 0.4762]], grad_fn=<SoftmaxBackward0>))


IndexError: tuple index out of range

In [ ]:
output = model(test_pairs)[1]
# print(output)



tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [1., 0.],
        [0., 1.],
        [0., 1.]], grad_fn=<RoundBackward0>)
tensor([[0., 1.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [0., 1.],
        [0., 1.]])


In [ ]:
error = 0

for i in range(1000):
    if torch.argmax(output[i]) != torch.argmax(test_target[i]):
        error += 1

print(error)

478
